In [2]:
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/crawler/academia/academia-pdf.json

In [3]:
import json
import cleaning
from tqdm import tqdm

In [4]:
with open('academia-pdf.json') as fopen:
    pdf = json.load(fopen)
    
len(pdf)

224

In [5]:
from unidecode import unidecode

def clean(string):
    string = [cleaning.cleaning(s) for s in string]
    
    string = [s.strip() for s in string if 'tarikh' not in s.lower() and 'soalan no' not in s.lower()]
    string = [s for s in string if not ''.join(s.split()[:1]).isdigit() and '.soalan' not in s.lower() and 'jum ' not in s.lower()]
    string = [s for s in string if not s[:3].isdigit() and not s[-3:].isdigit()]
    return string

In [6]:
outer = []

for k in tqdm(range(len(pdf))):

    c = clean(pdf[k]['content']['content'].split('\n'))
    t, last = [], 0

    i = 0
    while i < len(c):
        text = c[i]

        if len(text) > 5:
            if len(text.split()) > 1:
                t.append(text)
            last = i
        else:
            if len(t) and (i - last) > 2:
                t.append('')
                outer.extend(t)
                t = []
                last = i
            elif not len(t):
                last = i

        i += 1
    
    if len(t):
        t.append('')
        outer.extend(t)

100%|██████████| 224/224 [00:32<00:00,  6.88it/s]


In [7]:
len(outer)

651957

In [8]:
%%time

temp_vocab = list(set(cleaning.multiprocessing(outer, cleaning.unique_words)))

CPU times: user 390 ms, sys: 263 ms, total: 653 ms
Wall time: 806 ms


In [9]:
%%time

# important
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.duplicate_dots_marks_exclamations, list_mode = False)
print(len(temp_dict))

881
CPU times: user 102 ms, sys: 120 ms, total: 221 ms
Wall time: 723 ms


In [10]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 279404.76it/s]


In [11]:
%%time

# important
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_underscore, list_mode = False)
print(len(temp_dict))

71
CPU times: user 114 ms, sys: 116 ms, total: 230 ms
Wall time: 323 ms


In [12]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 257353.34it/s]


In [13]:
%%time

# important
temp_dict = cleaning.multiprocessing(outer, cleaning.isolate_spamchars, list_mode = False)
print(len(temp_dict))

0
CPU times: user 216 ms, sys: 148 ms, total: 364 ms
Wall time: 1.08 s


In [14]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_short_words, list_mode = False)
print(len(temp_dict))

2631
CPU times: user 101 ms, sys: 133 ms, total: 234 ms
Wall time: 280 ms


In [15]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 261864.04it/s]


In [16]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_long_words, list_mode = False)
print(len(temp_dict))

765
CPU times: user 89 ms, sys: 119 ms, total: 208 ms
Wall time: 263 ms


In [17]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 275939.03it/s]


In [18]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_ending_underscore, list_mode = False)
print(len(temp_dict))

10
CPU times: user 103 ms, sys: 103 ms, total: 206 ms
Wall time: 258 ms


In [19]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 261560.59it/s]


In [20]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_starting_underscore, list_mode = False)
print(len(temp_dict))

56
CPU times: user 83.7 ms, sys: 128 ms, total: 212 ms
Wall time: 264 ms


In [21]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 266383.28it/s]


In [22]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.end_punct, list_mode = False)
print(len(temp_dict))

138699
CPU times: user 570 ms, sys: 151 ms, total: 722 ms
Wall time: 800 ms


In [23]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 240032.70it/s]


In [24]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.start_punct, list_mode = False)
print(len(temp_dict))

37464
CPU times: user 224 ms, sys: 136 ms, total: 360 ms
Wall time: 445 ms


In [25]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 228724.65it/s]


In [26]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.join_dashes, list_mode = False)
print(len(temp_dict))

65
CPU times: user 106 ms, sys: 143 ms, total: 249 ms
Wall time: 347 ms


In [27]:
outer = cleaning.string_dict_cleaning(outer, temp_dict)

100%|██████████| 651957/651957 [00:02<00:00, 245834.14it/s]


In [28]:
results, result = [], []
for i in tqdm(outer):
    if not len(i) and len(result):
        results.append(result)
        result = []
    else:
        result.append(i)
        
if len(result):
    results.append(result)

100%|██████████| 651957/651957 [00:00<00:00, 1271085.76it/s]


In [30]:
import re

alphabets = '([A-Za-z])'
prefixes = (
    '(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt|Puan|puan|Tuan|tuan|sir|Sir)[.]'
)
suffixes = '(Inc|Ltd|Jr|Sr|Co|Mo)'
starters = '(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever|Dia|Mereka|Tetapi|Kita|Itu|Ini|Dan|Kami|Beliau|Seri|Datuk|Dato|Datin|Tuan|Puan)'
acronyms = '([A-Z][.][A-Z][.](?:[A-Z][.])?)'
websites = '[.](com|net|org|io|gov|me|edu|my)'
another_websites = '(www|http|https)[.]'
digits = '([0-9])'
before_digits = '([Nn]o|[Nn]ombor|[Nn]umber|[Kk]e|=)'
month = '([Jj]an(?:uari)?|[Ff]eb(?:ruari)?|[Mm]a(?:c)?|[Aa]pr(?:il)?|Mei|[Jj]u(?:n)?|[Jj]ula(?:i)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]kt(?:ober)?|[Nn]ov(?:ember)?|[Dd]is(?:ember)?)'


def split_into_sentences(text, minimum_length = 5):
    text = text.replace('\x97', '\n')
    text = '. '.join([s for s in text.split('\n') if len(s)])
    text = text + '.'
    text = unidecode(text)
    text = ' ' + text + '  '
    text = text.replace('\n', ' ')
    text = re.sub(prefixes, '\\1<prd>', text)
    text = re.sub(websites, '<prd>\\1', text)
    text = re.sub(another_websites, '\\1<prd>', text)
    text = re.sub('[,][.]+', '<prd>', text)
    if '...' in text:
        text = text.replace('...', '<prd><prd><prd>')
    if 'Ph.D' in text:
        text = text.replace('Ph.D.', 'Ph<prd>D<prd>')
    text = re.sub('[.]\s*[,]', '<prd>,', text)
    text = re.sub(before_digits + '\s*[.]\s*' + digits, '\\1<prd>\\2', text)
    text = re.sub(month + '[.]\s*' + digits, '\\1<prd>\\2', text)
    text = re.sub('\s' + alphabets + '[.][ ]+', ' \\1<prd> ', text)
    text = re.sub(acronyms + ' ' + starters, '\\1<stop> \\2', text)
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]' + alphabets + '[.]',
        '\\1<prd>\\2<prd>\\3<prd>',
        text,
    )
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]', '\\1<prd>\\2<prd>', text
    )
    text = re.sub(' ' + suffixes + '[.][ ]+' + starters, ' \\1<stop> \\2', text)
    text = re.sub(' ' + suffixes + '[.]', ' \\1<prd>', text)
    text = re.sub(' ' + alphabets + '[.]', ' \\1<prd>', text)
    text = re.sub(digits + '[.]' + digits, '\\1<prd>\\2', text)
    if '”' in text:
        text = text.replace('.”', '”.')
    if '"' in text:
        text = text.replace('."', '".')
    if '!' in text:
        text = text.replace('!"', '"!')
    if '?' in text:
        text = text.replace('?"', '"?')
    text = text.replace('.', '.<stop>')
    text = text.replace('?', '?<stop>')
    text = text.replace('!', '!<stop>')
    text = text.replace('<prd>', '.')
    sentences = text.split('<stop>')
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences if len(s) > minimum_length]
    return sentences

split_into_sentences('733 ke . 633 , berlaku penurunan akibat kesan program PMI .')

['733 ke.633 , berlaku penurunan akibat kesan program PMI .']

In [31]:
import malaya
import re

def strip(string):
    string = ' '.join(string)
    string = re.sub(r'[ ]+', ' ', string.replace('\n', ' ').replace('\t', ' ')).strip()
    return split_into_sentences(string)

In [32]:
output = []

for r in tqdm(results):
    output.extend(strip(r) + [''])

100%|██████████| 55538/55538 [00:22<00:00, 2506.92it/s]


In [33]:
len(output)

408807

In [34]:
output[:100]

['JABATAN USULUDDIN DAN FALSAFAH.',
 '',
 'KOLOKIUM SISWAZAH JABATAN USULUDDIN DAN FALSAFAH.',
 '',
 'Mazlan Ibrahim Nurulfathonah Mohd Effendy Nur Fathiah A Rozak Muhammad Akmal R .',
 'Azmi Zulkeffly Adimun Amir Sharifuddin Ahmad.',
 '',
 'JABATAN USULUDDIN DAN FALSAFAH FAKULTI PENGAJIAN ISLAM UNIVERSITI KEBANGSAAN MALAYSIA.',
 '',
 'Hak cipta / Copyright.',
 '',
 'Hak cipta terpelihara .',
 'Tiada bahagian daripada ter ini boleh diter semula , disimpan untuk pengeluaran atau ditukarkan ke dalam sebarang bentuk atau dengan sebarang alat juga pun , sama ada dengan cara elektronik , gambar serta rakaman dan sebagainya tanpa kebenaran bertulis daripada Jabatan Usuluddin dan Falsafah , FPI , UKM terlebih dahulu .',
 '',
 'All rights reserved .',
 'No part of this publication may be reproduced or transmitted in any form or by any means , electronic or mechanical , including photocopy , recording , or any information storage and retrieval system , without permission in writing from the Dep

In [35]:
with open('dumping-pdf.txt', 'w') as fopen:
    fopen.write('\n'.join(output))